# Training pipeline

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
%load_ext autoreload
%autoreload 2

## Create train & test data generator

### 1 - Generate csv files

In [ ]:
import os
import pandas as pd
import numpy as np
from data_utils import create_instance_csv

In [ ]:
dataset_path = '/root/data/aquabyte-images/erko/'
annotations_dir = 'instance_labels'
frames_dir = 'frames'

Create csv base file from instance masks

In [ ]:
# create_instance_csv(dataset_path, annotations_dir, frames_dir,
#                    target_path='/root/data/erko/')

Split train & test

In [ ]:
#dataset = pd.read_csv('/root/data/erko/annotations.csv', header=None)
#split = 0.8

In [ ]:
#msk = np.random.rand(len(dataset)) < split
#train_dataset = dataset[msk]
#train_dataset.to_csv('/root/data/erko/annotations_train.csv', header=None)
#test_dataset = dataset[~msk]
#test_dataset.to_csv('/root/data/erko/annotations_test.csv', header=None)

### 2 - Data generators

In [ ]:
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

In [ ]:
from keras_retinanet.utils.transform import random_transform_generator

In [ ]:
csv_train_data_file = '/root/data/aquabyte-images/erko/annotations_train.csv'
csv_test_data_file = '/root/data/aquabyte-images/erko/annotations_test.csv'
classID_file = '/root/data/aquabyte-images/erko/classID.csv'
batch_size = 4
# transform_generator = random_transform_generator(flip_x_chance=0.5)
transform_generator = None

In [ ]:
train_generator = CSVGenerator(
        csv_train_data_file,
        classID_file,
        transform_generator=transform_generator,
        batch_size=batch_size,
        image_min_side=800,
        image_max_side=1500
    )
test_generator = CSVGenerator(
       csv_test_data_file,
       classID_file,
       transform_generator=transform_generator,
       batch_size=batch_size,
       image_min_side=800,
       image_max_side=1500
   )

### 3 - Visualise a random input

In [ ]:
# TODO

## Model

In [ ]:
from keras_retinanet import models
from keras_retinanet.models.retinanet import retinanet_bbox
from keras_retinanet.bin.train import create_models

In [ ]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

### 1 - Create model & load weights

In [ ]:
backbone_name = 'resnet50'
freeze_backbone = False

In [ ]:
backbone = models.backbone(backbone_name)
weights = backbone.download_imagenet()
model, training_model, prediction_model = create_models(
            backbone_retinanet=backbone.retinanet,
            num_classes=train_generator.num_classes(),
            multi_gpu=1,
            weights=weights,
            freeze_backbone=freeze_backbone
        )

In [ ]:
import keras
from keras_retinanet.losses import smooth_l1, focal
from custom_metrics import jaccard_coef

In [ ]:
training_model.compile(
        loss={
            'regression'    : smooth_l1(),
            'classification': focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001))

In [ ]:
print(model.summary())

### 2 - Callbacks 

In [ ]:
from custom_callbacks import step_decay, SaveHistory, MAP_eval
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback

In [ ]:
saveh = SaveHistory('./erko_5k_0909.json')

In [ ]:
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# save model
filepath = os.path.join('/root/data/models/erko/detection/', 'retinanet_5k_0909_{epoch:02d}.h5')
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             save_best_only=True, 
                             mode='min')

In [ ]:
map_metric = MAP_eval(test_generator)

In [ ]:
train_generator.size()

## Training

In [ ]:
# start training
history = training_model.fit_generator(
        generator=train_generator,
        steps_per_epoch=train_generator.size()//batch_size,
        epochs=50,
        verbose=1,
        validation_data= test_generator,
        validation_steps= test_generator.size() // batch_size,
        callbacks=[lr_scheduler, saveh, checkpoint]
    )

Loss plot

In [ ]:
import json
import matplotlib.pyplot as plt
history = json.load(open('./erko_0907.json'))
plt.plot(history['loss'], label='train_loss')
plt.plot(history['val_loss'], label='val_loss')
plt.legend()

## Evaluation

In [ ]:
model_in = '/root/data/models/gopro/detection/weight_retinanet/new_go_pro/detection_19.h5'

In [ ]:
model = models.load_model(model_in, convert=True, backbone_name=backbone_name)

In [ ]:
from keras_retinanet.utils.eval import evaluate
# Threshold score to filter detections with
iou_threshold = 0.6
score_threshold = 0.2
max_detections = 30
save_path = '/root/data/models/gopro/detection/results_retinanet'

In [ ]:
def get_detections(generator, model, score_threshold=0.05, max_detections=100, save_path=None):
    """ Get the detections from the model using the generator.

    The result is a list of lists such that the size is:
        all_detections[num_images][num_classes] = detections[num_detections, 4 + num_classes]

    # Arguments
        generator       : The generator used to run images through the model.
        model           : The model to run on the images.
        score_threshold : The score confidence threshold to use.
        max_detections  : The maximum number of detections to use per image.
        save_path       : The path to save the images with visualized detections to.
    # Returns
        A list of lists containing the detections for each image in the generator.
    """
    all_detections = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in range(generator.size()):
        raw_image    = generator.load_image(i)
        image        = generator.preprocess_image(raw_image.copy())
        image, scale = generator.resize_image(image)

        # run network
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))[:3]

        # correct boxes for image scale
        boxes /= scale

        # select indices which have a score above the threshold
        indices = np.where(scores[0, :] > score_threshold)[0]

        # select those scores
        scores = scores[0][indices]

        # find the order with which to sort the scores
        scores_sort = np.argsort(-scores)[:max_detections]

        # select detections
        image_boxes      = boxes[0, indices[scores_sort], :]
        image_scores     = scores[scores_sort]
        image_labels     = labels[0, indices[scores_sort]]
        image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)

        #if save_path is not None:
            #draw_annotations(raw_image, generator.load_annotations(i), label_to_name=generator.label_to_name)
            #draw_detections(raw_image, image_boxes, image_scores, image_labels, label_to_name=generator.label_to_name)

            #cv2.imwrite(os.path.join(save_path, '{}.png'.format(i)), raw_image)

        # copy detections to all_detections
        for label in range(generator.num_classes()):
            all_detections[i][label] = image_detections[image_detections[:, -1] == label, :-1]


    return all_detections


def get_annotations(generator):
    """ Get the ground truth annotations from the generator.

    The result is a list of lists such that the size is:
        all_detections[num_images][num_classes] = annotations[num_detections, 5]

    # Arguments
        generator : The generator used to retrieve ground truth annotations.
    # Returns
        A list of lists containing the annotations for each image in the generator.
    """
    all_annotations = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in range(generator.size()):
        # load the annotations
        annotations = generator.load_annotations(i)

        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            all_annotations[i][label] = annotations[annotations[:, 4] == label, :4].copy()


    return all_annotations

In [ ]:
from keras_retinanet.utils.visualization import draw_detections, draw_annotations

In [ ]:
ll_detections     = get_detections(test_generator, model, score_threshold=score_threshold, max_detections=max_detections, save_path=save_path)

In [ ]:
all_annotations    = get_annotations(test_generator)

In [ ]:
all_annotations[0][1]

In [ ]:
generator = test_generator

In [ ]:
ll_detections[0][0]

In [ ]:
test_generator.size()

In [ ]:
i = 0

In [ ]:
annotations = test_generator.load_annotations(i)

In [ ]:
raw_image    = generator.load_image(i)
image        = generator.preprocess_image(raw_image.copy())
image, scale = generator.resize_image(image)
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))[:3]

In [ ]:
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))[:3]

        # correct boxes for image scale
        boxes /= scale

        # select indices which have a score above the threshold
        indices = np.where(scores[0, :] > score_threshold)[0]

        # select those scores
        scores = scores[0][indices]

        # find the order with which to sort the scores
        scores_sort = np.argsort(-scores)[:max_detections]

        # select detections
        image_boxes      = boxes[0, indices[scores_sort], :]
        image_scores     = scores[scores_sort]
        image_labels     = labels[0, indices[scores_sort]]
        image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)

In [ ]:
test_

In [ ]:
present_classes = 0
precision = 0
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations),
          test_data_generator.label_to_name(label), 'with average precision: {:.4f}'.format(average_precision))
    if num_annotations > 0:
        present_classes += 1
        precision       += average_precision
print('mAP: {:.4f}'.format(precision / present_classes))

## Generate json

In [ ]:
from result_generator import get_detections, get_annotations

In [ ]:
detections = get_detections(
    generator=test_data_generator, 
    model=model, 
    score_threshold=0., 
    max_detections=max_detections
)

In [ ]:
annotations = get_annotations(test_data_generator)

In [ ]:
annotations[0].keys()

In [ ]:
annotations[0][1].shape, detections[0][1].shape

In [ ]:
test_data_generator.image_names[0]

In [ ]:
import json

In [ ]:
with open('/root/data/blender_v4/training/test_low_rez/labels.json') as json_file:  
    data = json.load(json_file)

In [ ]:
data[0

In [ ]:
len(annotations)

In [ ]:
test_data_generator.size()

In [ ]:
f = open('test.txt', 'w')
f.write('hello \nlol \n')
f.close()

In [ ]:
i = 0

In [ ]:
tes